In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import statistics as s

In [2]:
from functions import log

In [3]:
df = pd.read_csv('files/file_list.txt', delimiter= '\t')

df = df[df["Article Citation"].str.startswith("PLoS")]
df

,Article File,Article Citation,AccessionID,LastUpdated (YYYY-MM-DD HH:MM:SS),PMID,License,Retracted
0,PMC000xxxxxx/PMC176545.xml,PLoS Biol. 2003 Oct 18; 1(1):e5,PMC176545,2022-01-13 00:01:34,12929205,CC BY,no
1,PMC000xxxxxx/PMC176546.xml,PLoS Biol. 2003 Oct 18; 1(1):e6,PMC176546,2022-01-13 00:01:35,12929206,CC BY,no
2,PMC000xxxxxx/PMC176547.xml,PLoS Biol. 2003 Oct 18; 1(1):e7,PMC176547,2022-01-13 00:01:35,0,CC BY,no
3,PMC000xxxxxx/PMC176548.xml,PLoS Biol. 2003 Oct 18; 1(1):e11,PMC176548,2022-01-13 00:01:34,0,CC BY,no
4,PMC000xxxxxx/PMC193604.xml,PLoS Biol. 2003 Oct 15; 1(1):e13,PMC193604,2022-01-13 00:01:34,12975658,CC BY,no
...,...,...,...,...,...,...,...
2607,PMC000xxxxxx/PMC549603.xml,PLoS Med. 2005 Feb 22; 2(2):e57,PMC549603,2022-01-13 02:39:26,0,CC BY,no
2608,PMC000xxxxxx/PMC549605.xml,PLoS Med. 2005 Mar 22; 2(3):e69,PMC549605,2022-01-13 02:10:07,15737013,CC BY,no
2609,PMC000xxxxxx/PMC549606.xml,PLoS Med. 2005 Mar 22; 2(3):e73,PMC549606,2022-01-13 02:09:26,15737014,CC BY,no
2610,PMC000xxxxxx/PMC549607.xml,PLoS Med. 2005 Mar 22; 2(3):e74,PMC549607,2022-01-13 02:39:25,0,CC0,no


In [4]:
# initialize empty lists
titles = []
abstracts = []
refs = []

In [5]:
def main(article_file):
    tree = ET.parse('./files/'+article_file) # parse file
    root = tree.getroot() # get root
    
    # get title
    title_element = root.findall('front')[0].findall('article-meta')[0].findall('title-group')[0].findall('article-title')[0]
    title = "".join(title_element.itertext())
    title = title.replace('\n','')
    
    # get abstract
    try:
        abstract_element = root.findall('front')[0].findall('article-meta')[0].findall('abstract')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    except: # no abstract, take body?
        abstract_element = root.findall('body')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    
    # get references
    pmid_list = []
    try:
        reference_element = root.findall('back')[0].findall('ref-list')[0].findall('ref')
        counter = 0
        for ref in reference_element:
            try:
                pmid = "".join(ref.findall('element-citation')[0].findall('pub-id')[0].itertext())
                pmid_list.append(pmid)
            except: # no pmid
                pass
    except:
        pmid_list = "NA"
        
    titles.append(title)
    abstracts.append(abstract)
    refs.append(pmid_list)
    
    log(f'Ran select_files.ipynb succesfully')
        

In [6]:
for i in df["Article File"]:
    main(i)

In [7]:
df["title"] = titles
df["abstract"] = abstracts
df["references"] = refs

In [8]:
df

,Article File,Article Citation,AccessionID,LastUpdated (YYYY-MM-DD HH:MM:SS),PMID,License,Retracted,title,abstract,references
0,PMC000xxxxxx/PMC176545.xml,PLoS Biol. 2003 Oct 18; 1(1):e5,PMC176545,2022-01-13 00:01:34,12929205,CC BY,no,The Transcriptome of the Intraerythrocytic Dev...,Plasmodium falciparum is the causative agent o...,"[12427465, 12351791, 12519984, 11123685, 11475..."
1,PMC000xxxxxx/PMC176546.xml,PLoS Biol. 2003 Oct 18; 1(1):e6,PMC176546,2022-01-13 00:01:35,12929206,CC BY,no,DNA Analysis Indicates That Asian Elephants Ar...,The origin of Borneo's elephants is controvers...,"[10739137, 8978083, 10762406, 12721223, 116817..."
2,PMC000xxxxxx/PMC176547.xml,PLoS Biol. 2003 Oct 18; 1(1):e7,PMC176547,2022-01-13 00:01:35,0,CC BY,no,Borneo Elephants: A High Priority for Conserva...,A new study settles a long-standing dispute ab...,NA
3,PMC000xxxxxx/PMC176548.xml,PLoS Biol. 2003 Oct 18; 1(1):e11,PMC176548,2022-01-13 00:01:34,0,CC BY,no,Monitoring Malaria: Genomic Activity of the Pa...,"Every year, malaria kills as many as 2.5 milli...",NA
4,PMC000xxxxxx/PMC193604.xml,PLoS Biol. 2003 Oct 15; 1(1):e13,PMC193604,2022-01-13 00:01:34,12975658,CC BY,no,Drosophila Free-Running Rhythms Require Interc...,Robust self-sustained oscillations are a ubiqu...,"[9630223, 11520929, 12839998, 10684876, 963542..."
...,...,...,...,...,...,...,...,...,...,...
2607,PMC000xxxxxx/PMC549603.xml,PLoS Med. 2005 Feb 22; 2(2):e57,PMC549603,2022-01-13 02:39:26,0,CC BY,no,Equitable Allocation of Antiretrovirals in Res...,Antiretroviral drugs change the lives of patie...,NA
2608,PMC000xxxxxx/PMC549605.xml,PLoS Med. 2005 Mar 22; 2(3):e69,PMC549605,2022-01-13 02:10:07,15737013,CC BY,no,Designing an Equitable Strategy for Allocating...,A study by Wilson and Blower in the February i...,"[15737005, 12399216, 15567015]"
2609,PMC000xxxxxx/PMC549606.xml,PLoS Med. 2005 Mar 22; 2(3):e73,PMC549606,2022-01-13 02:09:26,15737014,CC BY,no,Acquired Resistance of Lung Adenocarcinomas to...,BackgroundLung adenocarcinomas from patients w...,"[15118073, 15118125, 15329413, 15623594, 15604..."
2610,PMC000xxxxxx/PMC549607.xml,PLoS Med. 2005 Mar 22; 2(3):e74,PMC549607,2022-01-13 02:39:25,0,CC0,no,How Tumor Cells Acquire Resistance to Kinase I...,Acquired resistance to chemotherapy is a major...,NA


In [9]:
lens = df['abstract'].str.split().apply(len)
print(max(lens))
print(min(lens))
print(s.mean(lens))
print(s.stdev(lens))

2045
1
283.02377414561664
253.49620083683863


In [10]:
df.to_csv(r'files/select_files.txt', sep='\t', index = False)